In [1]:
import boto3
sess = boto3.Session()
s3 = boto3.resource('s3')

In [2]:
ecr_repo_name = 'eks-ml-containers'

region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')
image_cpu = '{}.dkr.ecr.{}.amazonaws.com/{}:latest-cpu'.format(account, region, ecr_repo_name)
image_gpu = '{}.dkr.ecr.{}.amazonaws.com/{}:latest-gpu'.format(account, region, ecr_repo_name)

In [3]:
!aws ecr create-repository --repository-name {ecr_repo_name}


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'eks-ml-containers' already exists in the registry with id '453691756499'


In [4]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com
!docker build -t {ecr_repo_name}:latest-cpu -f Dockerfile.cpu .
!docker build -t {ecr_repo_name}:latest-gpu -f Dockerfile.gpu .

WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Sending build context to Docker daemon  122.9kB
Step 1/3 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.4.1-cpu-py37-ubuntu18.04
 ---> b9f94444f73b
Step 2/3 : COPY tf2_cifar10.py /opt/ml/code/tf2_cifar10.py
 ---> Using cache
 ---> 9d14bbfc9e2e
Step 3/3 : ENTRYPOINT ["python", "/opt/ml/code/tf2_cifar10.py"]
 ---> Using cache
 ---> b2afe528ecbc
Successfully built b2afe528ecbc
Successfully tagged eks-ml-containers:latest-cpu
Sending build context to Docker daemon  122.9kB
Step 1/3 : FROM 763104351884.dkr.ecr.us-west-2.amazonaws.com/tensorflow-training:2.4.1-gpu-py37-cu110-ubuntu18.04
 ---> 8467bc1c5070
Step 2/3 : COPY tf2_cifar10.py /opt/ml/code/tf2_cifar10.py
 ---> Using cache
 ---> 3f82f579022e
Step 3/3 : ENTRYPOINT ["python

In [5]:
!$(aws ecr get-login --no-include-email --region {region})

!docker tag {ecr_repo_name}:latest-cpu {image_cpu}
!docker push {image_cpu}

!docker tag {ecr_repo_name}:latest-gpu {image_gpu}
!docker push {image_gpu}

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [453691756499.dkr.ecr.us-west-2.amazonaws.com/eks-ml-containers]

8ffe7374: Preparing 
776c5349: Preparing 
9e3bc3fc: Preparing 
b299b2c6: Preparing 
27dfc916: Preparing 
876691d9: Preparing 
0c18afaf: Preparing 
6f73a491: Preparing 
7bdd3d27: Preparing 
5a895314: Preparing 
0d3ec004: Preparing 
f2893f29: Preparing 
87643a94: Preparing 
26201183: Preparing 
fd80d0fa: Preparing 
9bd8d228: Preparing 
7ead7770: Preparing 
c6d2db45: Preparing 
bacb0351: Preparing 
e1acaabc: Layer already exists latest-cpu: digest: sha256:00bcde93629cae8491782f3f5c72aea71f4f0bdcf2145d63e4286a1016e431e2 size: 4509
The push refers to repository [453691756499.dkr.ecr.us-wes